In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

In [2]:
df = pd.read_csv("yelp_dataset.csv")

In [3]:
df.shape

(40000, 7)

In [4]:
df.columns

Index(['Unnamed: 0', 'label', 'reviews', 'review_after_removing_html_tags',
       'review_after_converted_emoji', 'review_after_removing_punctuation',
       'review_after_pos_and_lemmatization'],
      dtype='object')

In [5]:
df.isna().sum()

Unnamed: 0                            0
label                                 0
reviews                               0
review_after_removing_html_tags       0
review_after_converted_emoji          0
review_after_removing_punctuation     2
review_after_pos_and_lemmatization    4
dtype: int64

In [7]:
df.rename(columns={'label':'sentiment'}, inplace=True)

In [10]:
df["sentiment"] = df.sentiment.apply(lambda x: 0 if x == 1 else 1)

In [11]:
df[ df.sentiment == 1 ].shape

(20000, 7)

In [12]:
df[ df.sentiment == 0 ].shape

(20000, 7)

In [13]:
df.dropna(inplace=True)

In [14]:
df = df.reset_index()

In [16]:
tfidf_model = TfidfVectorizer()

X = tfidf_model.fit_transform(df["review_after_pos_and_lemmatization"])
y = df["sentiment"]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
y_train[ y_train == 1 ].shape

(16018,)

In [19]:
y_train[ y_train == 0 ].shape

(15978,)

In [20]:
y_test[ y_test == 1 ].shape

(3981,)

In [21]:
y_test[ y_test == 0 ].shape

(4019,)

#### KFold cross-validation

In [22]:
# Set up k-fold cross-validation
kf = KFold(n_splits=4, shuffle=True)

random_forest_scores = []
logistic_scores = []
svm_scores = []
gd_boost_scores = []

# Generate indices for each fold
for train_index, test_index in kf.split(X):
    x_train_data, x_test_data = X[train_index], X[test_index]
    y_train_data, y_test_data = y[train_index], y[test_index]

    ## Random Forest
    model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

    print("Fitting random forest to training data....")    
    model_forest = model_forest.fit(x_train_data, y_train_data)
    
    y_pred_data = model_forest.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    random_forest_scores.append(score)
    print(random_forest_scores)
    
    ## Logistic Regression
    
    model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
    print("Fitting Logistic Regression to training data....")  
    model_logistic_regression = model_logistic_regression.fit(x_train_data, y_train_data)
    
    y_pred_data = model_logistic_regression.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    logistic_scores.append(score)
    print(logistic_scores)
    
    ## SVM
    model_svm = SVC(C=0.1, kernel='linear')
    print("Fitting SVM to training data....")  
    model_svm = model_svm.fit(x_train_data, y_train_data)
    
    y_pred_data = model_svm.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    svm_scores.append(score)
    print(svm_scores)
    
    ## Gradient Boosting
    gb_classifier = GradientBoostingClassifier(random_state=2)
    print("Fitting Gradient Boosting to training data....")    
    gb_classifier.fit(x_train_data, y_train_data)
    
    y_pred_data = gb_classifier.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    gd_boost_scores.append(score)
    print(gd_boost_scores)
    
    print("=======================================================")

# print(f"Cross-validation scores: {scores}")
# print(f"Average score: {np.mean(scores)}")

Fitting random forest to training data....
[0.785978597859786]
Fitting Logistic Regression to training data....
[0.8504850485048505]
Fitting SVM to training data....
[0.8965896589658966]
Fitting Gradient Boosting to training data....
[0.8413841384138414]
Fitting random forest to training data....
[0.785978597859786, 0.7940794079407941]
Fitting Logistic Regression to training data....
[0.8504850485048505, 0.8534853485348535]
Fitting SVM to training data....
[0.8965896589658966, 0.903090309030903]
Fitting Gradient Boosting to training data....
[0.8413841384138414, 0.8466846684668466]
Fitting random forest to training data....
[0.785978597859786, 0.7940794079407941, 0.7853785378537854]
Fitting Logistic Regression to training data....
[0.8504850485048505, 0.8534853485348535, 0.8524852485248525]
Fitting SVM to training data....
[0.8965896589658966, 0.903090309030903, 0.8977897789778978]
Fitting Gradient Boosting to training data....
[0.8413841384138414, 0.8466846684668466, 0.840884088408840

In [23]:
print("Random Forest KFold cross-validation")
print(f"Cross-validation scores: {random_forest_scores}")
print(f"Average score: {np.mean(random_forest_scores)}\n")
print("Logistic Regression KFold cross-validation")
print(f"Cross-validation scores: {logistic_scores}")
print(f"Average score: {np.mean(logistic_scores)}\n")
print("SVM cross-validation")
print(f"Cross-validation scores: {svm_scores}")
print(f"Average score: {np.mean(svm_scores)}\n")
print("Gradient Boosting KFold cross-validation")
print(f"Cross-validation scores: {gd_boost_scores}")
print(f"Average score: {np.mean(gd_boost_scores)}\n")

Random Forest KFold cross-validation
Cross-validation scores: [0.785978597859786, 0.7940794079407941, 0.7853785378537854, 0.7708770877087708]
Average score: 0.7840784078407842

Logistic Regression KFold cross-validation
Cross-validation scores: [0.8504850485048505, 0.8534853485348535, 0.8524852485248525, 0.8502850285028503]
Average score: 0.8516851685168517

SVM cross-validation
Cross-validation scores: [0.8965896589658966, 0.903090309030903, 0.8977897789778978, 0.8988898889888989]
Average score: 0.8990899089908991

Gradient Boosting KFold cross-validation
Cross-validation scores: [0.8413841384138414, 0.8466846684668466, 0.8408840884088409, 0.8422842284228422]
Average score: 0.8428092809280928



#### Random Forest

In [24]:
# Start the timer
start_time = datetime.now()

model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

print("Fitting random forest to training data....")    
model_forest = model_forest.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Training Runtime:', runtime)

Fitting random forest to training data....
Random Forest Training Runtime: 0:01:06.418900


In [25]:
model_forest.score(X_test, y_test)

0.785625

In [26]:
model_forest.score(X_train, y_train)

0.8065383172896612

In [27]:
# Start the timer
start_time = datetime.now()

y_pred = model_forest.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Prediction Runtime:', runtime)

Random Forest Prediction Runtime: 0:00:01.049583


In [28]:
y_pred = cross_val_predict(model_forest, X_test, y_test)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.75      0.79      4019
           1       0.77      0.84      0.81      3981

    accuracy                           0.80      8000
   macro avg       0.80      0.80      0.80      8000
weighted avg       0.80      0.80      0.80      8000



In [30]:
print(confusion_matrix(y_test, y_pred))

[[3011 1008]
 [ 618 3363]]


In [31]:
filename = 'models/rd_tfidf_ml_model.sav'
pickle.dump(model_forest, open(filename, 'wb'))

#### Logistic Regression

In [32]:
# Start the timer
start_time = datetime.now()

model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

print("Fitting Logistic Regression to training data....")  
model_logistic_regression = model_logistic_regression.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Training Runtime:', runtime)

Fitting Logistic Regression to training data....
Logistic Regression Training Runtime: 0:00:00.269430


In [33]:
model_logistic_regression.score(X_test, y_test)

0.849875

In [34]:
model_logistic_regression.score(X_train, y_train)

0.8563570446305788

In [35]:
# Start the timer
start_time = datetime.now()

y_pred = model_logistic_regression.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Prediction Runtime:', runtime)

Logistic Regression Prediction Runtime: 0:00:00.015404


In [36]:
y_pred = cross_val_predict(model_logistic_regression, X_test, y_test)

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77      4019
           1       0.79      0.69      0.74      3981

    accuracy                           0.76      8000
   macro avg       0.76      0.76      0.76      8000
weighted avg       0.76      0.76      0.76      8000



In [38]:
print(confusion_matrix(y_test, y_pred))

[[3300  719]
 [1230 2751]]


In [39]:
filename = 'models/lg_tfidf_ml_model.sav'
pickle.dump(model_logistic_regression, open(filename, 'wb'))

#### SVM

In [40]:
# Start the timer
start_time = datetime.now()

model_svm = SVC(C=0.1, kernel='linear')

model_svm = model_svm.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:05:06.703104


In [41]:
model_svm.score(X_test, y_test)

0.898875

In [42]:
model_svm.score(X_train, y_train)

0.9180835104388049

In [43]:
# Start the timer
start_time = datetime.now()

y_pred = model_svm.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:01:06.474910


In [44]:
y_pred = cross_val_predict(model_svm, X_test, y_test)

In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87      4019
           1       0.87      0.85      0.86      3981

    accuracy                           0.86      8000
   macro avg       0.86      0.86      0.86      8000
weighted avg       0.86      0.86      0.86      8000



In [46]:
print(confusion_matrix(y_test, y_pred))

[[3511  508]
 [ 579 3402]]


In [47]:
filename = 'models/svm_tfidf_ml_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))

#### Gradient Boosting Classifier

In [48]:
# Start the timer
start_time = datetime.now()


# create a Gradient Boosting classifier with default hyperparameters
gb_classifier = GradientBoostingClassifier(random_state=2)

# fit the classifier to the training data
gb_classifier.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Gradient Boosting Prediction Runtime:', runtime)


Gradient Boosting Prediction Runtime: 0:08:42.678032


In [49]:
gb_classifier.score(X_test, y_test)

0.84

In [50]:
gb_classifier.score(X_train, y_train)

0.8572321540192525

In [51]:
# Start the timer
start_time = datetime.now()

y_pred = gb_classifier.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:00.022096


In [52]:
y_pred = cross_val_predict(gb_classifier, X_test, y_test)

In [53]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.84      0.84      4019
           1       0.84      0.82      0.83      3981

    accuracy                           0.83      8000
   macro avg       0.83      0.83      0.83      8000
weighted avg       0.83      0.83      0.83      8000



In [54]:
filename = 'models/gb_boosting_tfidf_ml_model.sav'
pickle.dump(gb_classifier, open(filename, 'wb'))